In [ ]:
['Revenue', 'Cost of revenue', 'Gross profit',
       'Research and development', 'Sales, General and administrative',
       'Total operating expenses', 'Operating income', 'Interest Expense',
       'Other income (expense)', 'Income before taxes',
       'Provision for income taxes', 'Net income', 'Basic_EPS',
       'Diluted_EPS', 'EBITDA', 'Total cash', 'Receivables',
       'Inventories', 'Prepaid expenses', 'Other current assets',
       'Total current assets', 'Gross property, plant and equipment',
       'Net property, plant and equipment', 'Goodwill',
       'Intangible assets', 'Other long-term assets',
       'Total non-current assets', 'Total assets', 'Short-term debt',
       'Accounts payable', 'Taxes payable', 'Accrued liabilities',
       'Other current liabilities', 'Total current liabilities',
       'Long-term debt', 'Deferred taxes liabilities',
       'Other long-term liabilities', 'Total non-current liabilities',
       'Total liabilities', 'Common stock', 'Additional paid-in capital',
       'Retained earnings', 'Treasury stock',
       "Total stockholders' equity",
       "Total liabilities and stockholders' equity", 'Capex to Revenue',
       'Cash per Share', 'EV to Free cash flow', 'EV to Sales',
       'Enterprise Value', 'Enterprise Value over EBITDA',
       'Free Cash Flow per Share', 'Graham Number', 'Income Quality',
       'Interest Debt per Share', 'Net Debt to EBITDA',
       'Net Income per Share', 'Operating Cash Flow per Share',
       'PB ratio', 'PE ratio', 'PTB ratio', 'Price to Sales Ratio',
       'R&D to Revenue', 'Revenue per Share',
       'Shareholders Equity per Share',
       'Stock-based compensation to Revenue',
       'Tangible Book Value per Share', 'stockprc', 'GDP', 'GNI',
       'Market_Cap', 'Average Inventory', 'Average Payables',
       'Average Receivables', 'Book Value per Share',
       'Days Payables Outstanding', 'Days of Inventory on Hand',
       'Inventory Turnover', 'Invested Capital', 'Market Cap',
       'Net Current Asset Value', 'Payables Turnover', 'Paid_LastYr?',
       'SectPaidLastYr', 'PropSectorAvgMktCap', 'currentRatio',
       'grossProfitMargin', 'returnOnAssets', 'PBratio', 'DistLast_1',
       'DistLast_3', 'DistLast_4', 'DistLast_5', 'DistLast_6'],
      dtype=object)

In [ ]:
['Revenue', 'Cost of revenue', 'Gross profit',
       'Research and development', 'Sales, General and administrative',
       'Total operating expenses', 'Operating income', 'Interest Expense',
       'Other income (expense)', 'Income before taxes',
       'Provision for income taxes', 'Net income', 'Basic_EPS',
       'Diluted_EPS', 'EBITDA', 'Total cash', 'Receivables',
       'Inventories', 'Prepaid expenses', 'Other current assets',
       'Total current assets', 'Gross property, plant and equipment',
       'Net property, plant and equipment', 'Goodwill',
       'Intangible assets', 'Other long-term assets',
       'Total non-current assets', 'Total assets', 'Short-term debt',
       'Accounts payable', 'Taxes payable', 'Accrued liabilities',
       'Other current liabilities', 'Total current liabilities',
       'Long-term debt', 'Deferred taxes liabilities',
       'Other long-term liabilities', 'Total non-current liabilities',
       'Total liabilities', 'Common stock', 'Additional paid-in capital',
       'Retained earnings', 'Treasury stock',
       'Accumulated other comprehensive income',
       "Total stockholders' equity",
       "Total liabilities and stockholders' equity", 'Capex to Revenue',
       'Cash per Share', 'Days Sales Outstanding', 'Debt to Assets',
       'EV to Free cash flow', 'EV to Sales', 'Earnings Yield',
       'Enterprise Value', 'Free Cash Flow Yield',
       'Free Cash Flow per Share', 'Graham Net-Net', 'Graham Number',
       'Income Quality', 'Intangibles to Total Assets',
       'Interest Coverage', 'Interest Debt per Share',
       'Net Debt to EBITDA', 'Net Income per Share',
       'Operating Cash Flow per Share', 'PB ratio', 'PE ratio',
       'PFCF ratio', 'PTB ratio', 'Price to Sales Ratio',
       'R&D to Revenue', 'Revenue per Share', 'SG&A to Revenue',
       'Shareholders Equity per Share',
       'Stock-based compensation to Revenue',
       'Tangible Book Value per Share', 'stockprc', 'mth6stockratio',
       'mth12stockratio', 'GDP', 'GNI', 'Market_Cap', 'Index Value',
       'Standard Error', 'Average Inventory', 'Average Payables',
       'Average Receivables', 'Book Value per Share',
       'Capex to Depreciation', 'Days Payables Outstanding',
       'Days of Inventory on Hand', 'Inventory Turnover',
       'Invested Capital', 'Market Cap', 'Net Current Asset Value',
       'Payables Turnover', 'Paid_LastYr?', 'Year', 'SectPaidLastYr',
       'PropSectorAvgMktCap', 'currentRatio', 'grossProfitMargin',
       'operatingProfitMargin', 'returnOnAssets', 'assetTurnover',
       'PBratio', 'Sector_Consumer Discretionary', 'Sector_Energy',
       'Sector_Health Care', 'Sector_Real Estate',
       'Sector_Telecommunication Services']